# Cardio

Jorge Galeano Maté

## Descripción

La OMS estima que cada año se producen 12 millones de muertes en todo el mundo debido
a enfermedades cardíacas, producidas en su gran mayoría por enfermedades
cardiovasculares. El pronóstico temprano de las enfermedades cardiovasculares puede
ayudar en la toma de decisiones sobre cambios en el estilo de vida en pacientes de alto
riesgo y reducir las complicaciones.
La base de datos cardio.csv contiene mediciones realizadas a pacientes hace 10 años, entre
los cuales se encuentran
- sex: M - F
- age: edad
- education: codificada, considerando que un valor más alto corresponde a un mayor
nivel educativo.
- currentSmoker: si el paciente es o no fumador actual
- cigsPerDay: el número de cigarrillos que la persona fumaba en promedio en un día.
- BPMeds: si el paciente estaba tomando medicamentos para la presión arterial (0: No,
1: Sí)
- prevalenStroke: si el paciente había tenido un accidente cerebrovascular previamente
(0: No, 1: Sí)
- prevalentHyp:: si el paciente era hipertenso o no (0: No, 1: Sí)
- diabetes: si el paciente tenía diabetes o no (0: No, 1: Sí)
- totChol: nivel de colesterol total
- sysBP: presión arterial sistólica
- diaBP_BMI: presión arterial diastólica
- BMI: Índice de masa corporal
- heartRate: : ritmo cardíaco
- glucose: nivel de glucosa
- TenYearCHD: se indica si el paciente sufrió o no una enfermedad coronaria en los
últimos 10 años.(0: No, 1: Sí)

## Tareas

1. Carga los datos y explóralos. Elimina los datos nulos o incorrectos
2. Escoge tres variables cuantitativas y realiza un análisis descriptivo de ellas,
utilizando indicadores y gráficos. ¿Cuál presenta mayor dispersión?
3. Elimina los datos atípicos del dataset. Para las siguientes preguntas, considera el
dataset "limpio"
4. ¿Qué variables cuantitativas presentan mayor correlación? Explica.
5. Si de este dataset se escoge un paciente al azar, ¿cuál es la probabilidad de que sea
hombre, si se sabe que pertenece al mayor cuartil de la variable correspondiente al
índice de masa corporal?
6. ¿Es razonable afirmar que, para una persona cualquiera (no necesariamente dentro
del dataset), su ritmo cardiaco promedio es 75? Explica y justifica.
7. Ser hombre, ¿influye en el promedio de cigarrillos consumidos por día, dentro de los
pacientes fumadores? Explica.
8. Construye un modelo de regresión lineal que permita relacionar 6 variables del
dataset con el índice de masa corporal. (Debes incluir al menos dos variables
cualitativas). Evalúa tu modelo y explica.
9. Construye un modelo de regresión logística para predecir el riesgo de sufrir una
enfermedad coronaria en los próximos diez años, a partir de las variables descritas.
Verifica el balanceo de datos y evalúa tu modelo.
10. Separa los modelos de regresión anteriores en dos distintos, respectivamente,
considerando alguna variable categórica. Compara y concluye.

----------

1. Cargar datos y explorarlos. Eliminar nulos o incorrectos.

In [ ]:
import pandas as pd

df = pd.read_csv("cardio.csv", sep=",")

df.head()

In [ ]:
for columna in df.columns:
    print(f"{columna}: {df[columna].unique()}")

In [ ]:
df.info()

In [ ]:
df["currentSmoker"] = df["currentSmoker"].replace("No", False)
df["currentSmoker"] = df["currentSmoker"].replace("Yes", True)

df["BPMeds"] = df["BPMeds"].replace(0, False)
df["BPMeds"] = df["BPMeds"].replace(1, True)
df["BPMeds"] = df["BPMeds"].fillna(False)

df["prevalentHyp"] = df["prevalentHyp"].replace(0, False)
df["prevalentHyp"] = df["prevalentHyp"].replace(1, True)

df["prevalentStroke"] = df["prevalentStroke"].replace(0, False)
df["prevalentStroke"] = df["prevalentStroke"].replace(1, True)

df["diabetes"] = df["diabetes"].replace(0, False)
df["diabetes"] = df["diabetes"].replace(1, True)

df["TenYearCHD"] = df["TenYearCHD"].replace(0, False)
df["TenYearCHD"] = df["TenYearCHD"].replace(1, True)

df.info()

In [ ]:
mediana_glucosa = df["glucose"].median()

df["glucose"] = df["glucose"].fillna(mediana_glucosa)

df = df.dropna()

In [ ]:
df.head()

In [ ]:
import altair as alt
import numpy as np

df_long = df.select_dtypes(include=np.number).melt(var_name="Variable", value_name="Valor")

alt.data_transformers.disable_max_rows()

alt.Chart(df_long, height=30, width=400).mark_boxplot()\
    .encode(x=alt.X('Valor:Q', title=None), row='Variable:N')\
    .resolve_scale(x='independent')\
    .interactive(True)

2. Escoger 3 variables cuantitativas y describirlas con indicadores y gráficos. ¿Cuál presenta mayor dispersión?

In [ ]:
df_long = df[["BMI", "age", "glucose"]].melt(var_name="Variable", value_name="Valor")

alt.data_transformers.disable_max_rows()

alt.Chart(df_long).mark_boxplot()\
    .encode(x=alt.X("Valor:Q"), row="Variable:N")\
    .resolve_scale(x="independent")\
    .interactive(True)

In [ ]:
print(f"""Dispersión según desviación estándar (std):
    BMI = {np.std(df["BMI"])}
    Edad = {np.std(df["age"])}
    Glucosa = {np.std(df["glucose"])}
""")

Aparentemente, en las gráficas se observa que la glucosa y el BMI presentan mucha dispersión, además de gran cantidad de datos atípicos.

El BMI ronda desde los 15.54 hasta 56.8, siendo el valor medio 25.36.

La edad no presenta datos atípicos y ronda desde 32 años hasta 70 años.

Por último, la glucosa ronda entre 40 y 394.

En base a los indicadores de dispersión, la glucosa presenta la mayor, seguida por la edad.

3. Eliminar datos atípicos.

In [ ]:
# ATENCION - Este bucle solo se debe ejecutar UNA VEZ. Más ejecuciones equivalen a borrado extra de datos.

for columna in df.select_dtypes(include=np.number).columns:
    IQR = df[columna].quantile(0.75) - df[columna].quantile(0.25)
    outlier_sup = df[columna].quantile(0.75) + 1.5*IQR
    outlier_inf = df[columna].quantile(0.25) - 1.5*IQR
    for dato in df[columna]:
        if dato > outlier_sup or dato < outlier_inf:
            df = df.drop(df[df[columna] == dato].index)

4. Variables cuantitativas con mayor correlación.

In [ ]:
df.select_dtypes(include=np.number).corr()

Las variables con mayor correlación son la presión arterial sistólica y diastólica, con un 0.76. Esto es lógico debido a que ambas expresan la presión arterial.

En segundo lugar, se encuentra la presión arterial sistólica y la edad, posiblemente relacionadas porque con la edad aumenta la presión arterial. Además, también hay relación entre el BMI y la presión arterial sistólica y diastólica, debido a que un mayor BMI puede indicar mayor masa grasa, lo que puede aumentar la presión arterial.

5. Si de este dataset se escoge un paciente al azar, ¿cuál es la probabilidad de que sea
hombre, si se sabe que pertenece al mayor cuartil de la variable correspondiente al
BMI?

In [ ]:
prob = len(df[(df["BMI"] >= df["BMI"].quantile(0.75)) & (df["sex"] == "M")]) / len(df) * 100

print(f"Probabilidad = {round(prob, 2)}%")

6. ¿Es razonable afirmar que, para una persona cualquiera (no necesariamente dentro del dataset), su ritmo cardiaco promedio es 75? Explica y justifica.

No, no es razonable afirmar que el ritmo cardíaco promedio es 75, ya que el ritmo cardíaco puede variar dependiendo de la persona, la edad, el estado físico, entre otros factores.

7. Ser hombre, ¿influye en el promedio de cigarrillos consumidos por día, dentro de los pacientes fumadores? Explica.

In [ ]:
alt.Chart(df[df["currentSmoker"] == True]).mark_bar()\
    .encode(x="cigsPerDay:Q", y="sex:N")\
    .resolve_scale(x="independent")

Como se observa en el gráfico, los hombres fuman significativamente más cigarrillos que las mujeres, por lo que se puede afirmar que ser hombre influye en el promedio de cigarrillos consumidos por día.

8. Construir un modelo de regresión lineal que permita relacionar 6 variables del dataset con el índice de masa corporal. Evaluar el modelo y explicar.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, mean_absolute_percentage_error, r2_score

x = df[["age", "education", "cigsPerDay", "totChol", "heartRate", "glucose"]]
y = df[["BMI"]]

model = LinearRegression()

model.fit(x, y)

y_pred = model.predict(x)

print(f"""
ERRORES:
    MAE = {mean_absolute_error(y, y_pred)}
    MSE = {mean_squared_error(y, y_pred)}
    RMSE = {root_mean_squared_error(y, y_pred)}
    MAPE = {mean_absolute_percentage_error(y, y_pred)}
    R2 = {r2_score(y, y_pred)}
""")

El modelo presenta un MAPE de 0.11, que indica un 11% de error en la predicción.

El coeficiente de determinación es de 0.048, que indica que puede explicar solamente un 4.8% de la variabilidad de la variable dependiente, por lo que no es un buen modelo.

9. Construir un modelo de regresión logística para predecir el riesgo de sufrir una enfermedad coronaria en los próximos diez años, a partir de las variables descritas. Verificar el balanceo de datos y evaluar el modelo.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

df_copy = df.copy()

df_copy[df_copy.select_dtypes(include=np.bool).columns] = df_copy[df_copy.select_dtypes(include=np.bool).columns].astype(int)

x = df_copy[["age", "education", "currentSmoker", "cigsPerDay", "BPMeds",
             "prevalentStroke", "prevalentHyp", "diabetes", "totChol",
             "sysBP", "diaBP", "BMI", "heartRate", "glucose"]]

y = df_copy["TenYearCHD"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

booleanos = ["currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes"]
no_booleanos = ["age", "education", "cigsPerDay", "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"]

x_train[no_booleanos] = scaler.fit_transform(x_train[no_booleanos])
x_test[no_booleanos] = scaler.transform(x_test[no_booleanos])

model = LogisticRegression(max_iter=10000)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

y_prob = model.predict_proba(x_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_prob))

A primera vista, el modelo presenta un 87% de exactitud, lo que puede indicar que es un buen modelo. Además, el AUC es de 0.73, lo cual le da una calificación aceptable.

No obstante, hay un grave desbalanceo de datos. Encontramos 605 casos negativos y solo 87 casos positivos. Esto implica que el modelo tiende a predecir más casos negativos que positivos, lo que puede ser un problema en la predicción de enfermedades coronarias.

Fijándonos en el F1-score de los casos positivos, encontramos un valor de 0,04, lo que indica que el modelo no es bueno para predecir casos positivos.

10. Separar los modelos de regresión anteriores en dos distintos, respectivamente, considerando alguna variable categórica. Comparar y concluir.

In [ ]:
df_hombres = df[df["sex"] == "M"]
df_mujeres = df[df["sex"] == "F"]

x = df_hombres[["age", "education", "cigsPerDay", "totChol", "heartRate", "glucose"]]
y = df_hombres[["BMI"]]

model = LinearRegression()

model.fit(x, y)

y_pred = model.predict(x)

print(f"""
ERRORES H:
    MAE = {mean_absolute_error(y, y_pred)}
    MSE = {mean_squared_error(y, y_pred)}
    RMSE = {root_mean_squared_error(y, y_pred)}
    MAPE = {mean_absolute_percentage_error(y, y_pred)}
    R2 = {r2_score(y, y_pred)}
""")

x = df_mujeres[["age", "education", "cigsPerDay", "totChol", "heartRate", "glucose"]]
y = df_mujeres[["BMI"]]

model = LinearRegression()

model.fit(x, y)

y_pred = model.predict(x)

print(f"""
ERRORES M:
    MAE = {mean_absolute_error(y, y_pred)}
    MSE = {mean_squared_error(y, y_pred)}
    RMSE = {root_mean_squared_error(y, y_pred)}
    MAPE = {mean_absolute_percentage_error(y, y_pred)}
    R2 = {r2_score(y, y_pred)}
""")

Realizando una separación por sexos, se observa que los errores varían. En el modelo que evalúa solo a los hombres, el MAPE es del 9,9%, mientras que en el modelo que evalúa solo a las mujeres, el MAPE es del 10,3%. En cambio, el coeficiente de determinación es de 3,9% para hombres y 11,4% para mujeres, lo que indica que el modelo de las mujeres explica una mayor variabilidad de los datos respecto al de los hombres y respecto al común.

In [ ]:
df_copy_hombres = df_hombres.copy()
df_copy_mujeres = df_mujeres.copy()

df_copy_hombres[df_copy_hombres.select_dtypes(include=np.bool).columns] = df_copy_hombres[df_copy_hombres.select_dtypes(include=np.bool).columns].astype(int)
df_copy_mujeres[df_copy_mujeres.select_dtypes(include=np.bool).columns] = df_copy_mujeres[df_copy_mujeres.select_dtypes(include=np.bool).columns].astype(int)

x = df_copy_hombres[["age", "education", "currentSmoker", "cigsPerDay", "BPMeds",
             "prevalentStroke", "prevalentHyp", "diabetes", "totChol",
             "sysBP", "diaBP", "BMI", "heartRate", "glucose"]]

y = df_copy_hombres["TenYearCHD"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

booleanos = ["currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes"]
no_booleanos = ["age", "education", "cigsPerDay", "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"]

x_train[no_booleanos] = scaler.fit_transform(x_train[no_booleanos])
x_test[no_booleanos] = scaler.transform(x_test[no_booleanos])

model = LogisticRegression(max_iter=10000)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

y_prob = model.predict_proba(x_test)[:, 1]

print("EVALUACIÓN HOMBRES:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_prob))

x = df_copy_mujeres[["age", "education", "currentSmoker", "cigsPerDay", "BPMeds",
             "prevalentStroke", "prevalentHyp", "diabetes", "totChol",
             "sysBP", "diaBP", "BMI", "heartRate", "glucose"]]

y = df_copy_mujeres["TenYearCHD"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

booleanos = ["currentSmoker", "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes"]
no_booleanos = ["age", "education", "cigsPerDay", "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"]

x_train[no_booleanos] = scaler.fit_transform(x_train[no_booleanos])
x_test[no_booleanos] = scaler.transform(x_test[no_booleanos])

model = LogisticRegression(max_iter=10000)

model.fit(x_train, y_train)

y_pred = model.predict(x_test)

y_prob = model.predict_proba(x_test)[:, 1]

print("EVALUACIÓN MUJERES:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))
print("AUC-ROC:", roc_auc_score(y_test, y_prob))

En el modelo que evalúa a los hombres, se obtiene una exactitud del 83% y un AUC de 0,7; y en el de las mujeres, se obtiene una exactitud del 92% y un AUC de 0,7.

En ambos casos, se observa un desbalanceo de datos, al igual que en el modelo común, con un mayor número de casos negativos que positivos, lo que afecta gravemente la predicción de enfermedades coronarias. En el mejor de los casos, encontramos un F1-score de 0,04 en el modelo de los hombres, lo cual nos indica que no es un modelo fiable.

### Conclusiones finales

Hemos podido evaluar datos sobre variables cuantitativas y cualitativas de salud. En la parte descriptiva, hemos encontrado una ligera correlación entre la presión arterial sistólica y la edad; así como entre el BMI y la presión arterial sistólica y diastólica. Además, hemos encontrado que ser hombre influye en el promedio de cigarrillos consumidos por día.

Por otra parte, en los modelos predictivos, no hemos encontrado un modelo que se adapte correctamente a los datos. En el modelo de regresión lineal, el coeficiente de determinación es muy bajo, lo que indica que no es un buen modelo. En el modelo de regresión logística, encontramos un desbalanceo de datos que afecta la predicción de enfermedades coronarias. En ambos casos, el F1-score es muy bajo, lo que indica que no son modelos fiables. Se requeriría de más datos o de un análisis más profundo para poder realizar una predicción más acertada.